# Logistic Regression with SPU

>The following codes are demos only. It's **NOT for production** due to system security concerns, please **DO NOT** use it directly in production.

[SPU](https://www.secretflow.org.cn/docs/spu/en/) is a domain specific compiler and runtime suite, which provides provable secure computation service. SPU compiler uses [XLA](https://www.tensorflow.org/xla) as its front-end IR, which supports diverse AI framework (like Tensorflow, JAX and PyTorch). SPU compiler translates XLA to an IR which could be interpreted by the SPU runtime. Currently SPU team highly recommends using [JAX](https://github.com/google/jax) as the frontend.

## Learning Objectives:

After doing this lab, you'll know how to:

* How to write a Logistic Regression trainning program with JAX.
* How to convert a JAX program to an SPU(MPC) program painlessly.

In this lab, we select [Breast Cancer](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(diagnostic)) as the dataset. We need to decide whether cancer is malignant or benign with 30 features. In the MPC program, two parties will train the model jointly and each party would provide half of features(15).

While, first, let's just forget MPC settings and just write a Logistic Regression training program with JAX directly.

## Train a model with JAX

### Load the Dataset


We are going to split the whole dataset into train and test subsets after normalization with `breast_cancer`.
* if `train` is `True`, returns train subsets. In order to simulate training with vertical dataset splitting, the `party_id` is provided.
* else, returns test subsets.

In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer


def breast_cancer(party_id=None, train: bool = True) -> (np.ndarray, np.ndarray):
    x, y = load_breast_cancer(return_X_y=True)
    x = (x - np.min(x)) / (np.max(x) - np.min(x))
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2, random_state=42
    )

    if train:
        if party_id:
            if party_id == 1:
                return x_train[:, :15], _
            else:
                return x_train[:, 15:], y_train
        else:
            return x_train, y_train
    else:
        return x_test, y_test

### Define the Model

First, let's define the loss function, which is a negative log-likelihood in our case.

In [2]:
import jax.numpy as jnp


def sigmoid(x):
    return 1 / (1 + jnp.exp(-x))


# Outputs probability of a label being true.
def predict(W, b, inputs):
    return sigmoid(jnp.dot(inputs, W) + b)


# Training loss is the negative log-likelihood of the training examples.
def loss(W, b, inputs, targets):
    preds = predict(W, b, inputs)
    label_probs = preds * targets + (1 - preds) * (1 - targets)
    return -jnp.mean(jnp.log(label_probs))

Second, let's define a single train step with SGD optimizer. Just to remind you, x1 represents 15 features from one party while x2 represents the other 15 features from the other party.

In [3]:
from jax import grad


def train_step(W, b, x1, x2, y, learning_rate):
    x = jnp.concatenate([x1, x2], axis=1)
    Wb_grad = grad(loss, (0, 1))(W, b, x, y)
    W -= learning_rate * Wb_grad[0]
    b -= learning_rate * Wb_grad[1]
    return W, b


Last, let's build everything together as a `fit` method which returns the model and losses of each epoch.

In [4]:
def fit(W, b, x1, x2, y, epochs=1, learning_rate=1e-2):
    for _ in range(epochs):
        W, b = train_step(W, b, x1, x2, y, learning_rate=learning_rate)
    return W, b

### Validate the Model

We could use the AUC to validate a binary classification model.

In [5]:
from sklearn.metrics import roc_auc_score


def validate_model(W, b, X_test, y_test):
    y_pred = predict(W, b, X_test)
    return roc_auc_score(y_test, y_pred)

### Have a try!

Let's put everything we have together and train a LR model!

In [6]:
%matplotlib inline

# Load the data
x1, _ = breast_cancer(party_id=1, train=True)
x2, y = breast_cancer(party_id=2, train=True)

# Hyperparameter
W = jnp.zeros((30,))
b = 0.0
epochs = 10
learning_rate = 1e-2

# Train the model
W, b = fit(W, b, x1, x2, y, epochs=10, learning_rate=1e-2)

# Validate the model
X_test, y_test = breast_cancer(train=False)
auc = validate_model(W, b, X_test, y_test)
print(f'auc={auc}')

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


auc=0.9878807730101539


Just remember the AUC here since we would like to do the similar thing with SPU!

## Train a Model with SPU

At this part, we are going to show you how to do the similar training with MPC securely!

### Init the Environment

We are going to init three virtual devices on our physical environment.
- alice, bob：Two PYU devices for local plaintext computation.
- spu：SPU device consists with alice and bob for MPC secure computation.

In [7]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))


# In case you have a running secretflow runtime already.
try:
    sf.shutdown()
except:
    pass  # Already shutdown or not initialized

sf.init(['alice', 'bob'], address='local')

alice, bob = sf.PYU('alice'), sf.PYU('bob')
spu = sf.SPU(sf.utils.testing.cluster_def(['alice', 'bob']))

The version of SecretFlow: 1.12.0b0


/opt/anaconda3/envs/limingbo_sf/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
/opt/anaconda3/envs/limingbo_sf/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2025-06-23 15:54:06,524	INFO worker.py:1841 -- Started a local Ray instance.


(pyu_fn pid=3446280) An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


(pyu_fn pid=3446280) [2025-06-23 15:54:09.374] [warning] [openssl_factory.cc:83] Yacl has been configured to use Yacl's entropy source, but unable to find one. Fallback to use openssl's default entropy srouce
(pyu_fn pid=3446280) [2025-06-23 15:54:09.378] [warning] [openssl_factory.cc:83] Yacl has been configured to use Yacl's entropy source, but unable to find one. Fallback to use openssl's default entropy srouce


(pyu_fn pid=3446268) 2025-06-23 15:54:11,258,258 INFO [xla_bridge.py:backends:863] Unable to initialize backend 'cuda': 
(pyu_fn pid=3446268) 2025-06-23 15:54:11,258,258 INFO [xla_bridge.py:backends:863] Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(pyu_fn pid=3446268) 2025-06-23 15:54:11,259,259 INFO [xla_bridge.py:backends:863] Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
(pyu_fn pid=3446268) 2025-06-23 15:54:11,259,259 WARNING [xla_bridge.py:_suggest_missing_backends:901] An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


(pyu_fn pid=3446278) [2025-06-23 15:54:11.335] [warning] [openssl_factory.cc:83] Yacl has been configured to use Yacl's entropy source, but unable to find one. Fallback to use openssl's default entropy srouce
(pyu_fn pid=3446278) [2025-06-23 15:54:11.337] [warning] [openssl_factory.cc:83] Yacl has been configured to use Yacl's entropy source, but unable to find one. Fallback to use openssl's default entropy srouce


(pyu_fn pid=3446278) 2025-06-23 15:54:11,322,322 INFO [xla_bridge.py:backends:863] Unable to initialize backend 'cuda': 
(pyu_fn pid=3446278) 2025-06-23 15:54:11,322,322 INFO [xla_bridge.py:backends:863] Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(pyu_fn pid=3446278) 2025-06-23 15:54:11,324,324 INFO [xla_bridge.py:backends:863] Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
(pyu_fn pid=3446278) 2025-06-23 15:54:11,324,324 WARNING [xla_bridge.py:_suggest_missing_backends:901] An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


### Load the Dataset

we instruct alice and bob to load the train subset respectively.

In [8]:
x1, _ = alice(breast_cancer)(party_id=1)
x2, y = bob(breast_cancer)(party_id=2)

x1, x2, y

(<secretflow.device.device.pyu.PYUObject at 0x7fc69c18bb50>,
 <secretflow.device.device.pyu.PYUObject at 0x7fc7b8757e20>)

Before training, we need to pass hyperparamters and all data to SPU device. SecretFlow provides two methods:
- secretflow.to: transfer a PythonObject or DeviceObject to a specific device.
- DeviceObject.to: transfer the DeviceObject to a specific device.

In [9]:
device = spu

W = jnp.zeros((30,))
b = 0.0

W_, b_, x1_, x2_, y_ = (
    sf.to(alice, W).to(device),
    sf.to(alice, b).to(device),
    x1.to(device),
    x2.to(device),
    y.to(device),
)

### Train the model

Now we are ready to train a LR model with SPU. After training, losses and model are SPUObjects which are still secret.

In [10]:
W_, b_ = device(
    fit,
    static_argnames=['epochs'],
    num_returns_policy=sf.device.SPUCompilerNumReturnsPolicy.FROM_USER,
    user_specified_num_returns=2,
)(W_, b_, x1_, x2_, y_, epochs=10, learning_rate=1e-2)

W_, b_

(<secretflow.device.device.spu.SPUObject at 0x7fc79c648190>,
 <secretflow.device.device.spu.SPUObject at 0x7fc79c6497e0>)

### Reveal the result

In order to check the trained model, we need to convert SPUObject(secret) to Python object(plaintext). SecretFlow provide `sf.reveal` to convert any DeviceObject to Python object.

> Be care with `sf.reveal`，since it may result in secret leak。

Finally, let's validate the model with AUC.

In [11]:
auc = validate_model(sf.reveal(W_), sf.reveal(b_), X_test, y_test)
print(f'auc={auc}')

auc=0.987880773010154


You may find the model from SPU training program achieve the same AUC as JAX program.

This is the end of lab.